## 데이터 전처리 코드
### 필요한 라이브러리 re, pandas, librosa, json, transformers

## 라벨 내 데이터 정제

In [2]:
import json
import re
import pandas as pd
from pathlib import Path

def load_label_data(label_dir):
    """
    label_dir 안의 모든 .json을 읽어 DataFrame 생성.
    - (NO:...) 는 무조건 삭제(묵음 표현, 노이즈 등)
    - 나머지 괄호는 안의 한글만 남김(한글이 없으면 괄호 통째로 제거)
    """
    data_list = []
    for json_file in Path(label_dir).glob('*.json'):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            text = data['발화정보']['stt']

            # 1) (NO: ...) 는 무조건 삭제
            text = re.sub(r'\(NO:[^)]*\)', '', text)

            # 2) 괄호 안에 한글이 있는 경우 → 한글만 남기기
            #    예: (SN: 안녕하세요123!) -> 안녕하세요
            text = re.sub(r'\([^)]*?([가-힣]+(?:\s+[가-힣]+)*)[^)]*?\)', r'\1', text)

            # 3) 여전히 남아있는 괄호(한글이 전혀 없던 경우) 통째로 제거
            text = re.sub(r'\([^)]*\)', '', text)

            # 4) 공백/탭/줄바꿈 정리
            text = re.sub(r'\s+', ' ', text).strip()

            data_list.append({
                'audio_path': data['발화정보']['fileNm'],
                'text': text
            })

    return pd.DataFrame(data_list)


In [3]:
train_df = load_label_data('/workspace/train/label')
test_df = load_label_data('/workspace/test/label')

#학습, 검증 데이터 작성

In [4]:
train_df

,audio_path,text
0,노인남여_노인대화07_M_1527672611_76_수도권_실내_07336.wav,이제 성당에 데리고 가잖아 근데 그것도 사실은
1,노인남여_노인대화07_F_1528838253_60_강원_실내_07687.wav,요즘은 그러니까 뭐 먹지 하면서도 내가 하기
2,노인남여_노인대화08_M_1529907003_68_강원_실내_08967.wav,참 우리나라 사람들이 캐나다에 가면
3,노인남여_노인대화07_F_1528838253_60_강원_실내_07356.wav,평일에 바쁘다가 토요일 일요일 되되면은 너무 힘들어요
4,노인남여_노인대화08_M_1521481375_64_강원_실내_09181.wav,그렇게 해서 봉지를 묶어 가지고 이렇게 흔들어주면
...,...,...
33815,노인남여_노인대화14_F_1528705879_62_충청_실내_18704.wav,잠바 정도는 항상 갖고 다녀야 하고 바람막이도요
33816,노인남여_노인대화07_F_1520511716_63_수도권_실내_08708.wav,그럼 몇 살까지 였습니까 우리가 육십 네살에 타면은
33817,노인남여_노인대화07_M_1527864598_70_강원_실내_07343.wav,불안하고 긴장되는 마음 알지
33818,노인남여_노인대화07_M_1527639004_76_강원_실내_08242.wav,어머님 세대는 건강과 뭐 음식 같은 거 신경을 안 쓴 세대예요


In [5]:
test_df

,audio_path,text
0,노인남여_노인대화08_M_1527659796_62_수도권_실내_09849.wav,말 끝마다 꼬치꼬치 대꾸하고 아빠한테 엉길라고 하고
1,노인남여_노인대화07_F_1520511716_63_수도권_실내_07286.wav,저번에 이야기 했듯이 애가 지금 삼 학년이랑 일 학년이잖아
2,노인남여_노인대화08_M_1527659796_62_수도권_실내_09630.wav,주전부리는 거 좋아하는데
3,노인남여_노인대화08_M_1528799473_66_강원_실내_09307.wav,날짜가 하루 딱 넘어가잖아요
4,노인남여_노인대화07_F_1527804469_64_강원_실내_08086.wav,수찬이는 동의보감이야 맨날 한방에 대해서 들어와서
...,...,...
8450,노인남여_노인대화07_F_1528705879_62_충청_실내_07281.wav,공휴일도 쉬니까 쉬는 날이 너무 많아
8451,노인남여_노인대화08_M_1528799473_66_강원_실내_10222.wav,이제 그림으로만 보고 사진으로만 봤는데
8452,노인남여_노인대화07_M_1527639004_76_강원_실내_08508.wav,어릴 때 주로 내 손으로 해 줬었는데 크고 나니까
8453,노인남여_노인대화08_M_1527525156_60_강원_실내_08812.wav,목욕탕을 하면 또 가족들이 전신에 목욕탕 하고


In [8]:
test_df.to_csv('./test_labels') #이후에도 사용할 수 있게 저장

In [2]:
import soundfile as sf
from pathlib import Path

audio_folder = Path("/workspace/test/data")
wrong_sr_files = []

for audio_file in audio_folder.rglob("*.wav"):
    try:
        info = sf.info(audio_file)
        sr = info.samplerate
        if sr != 16000:
            wrong_sr_files.append(str(audio_file))
    except Exception as e:
        print(f"오류 발생: {audio_file}, {e}")

print(f"샘플링 레이트가 16000이 아닌 파일 개수: {len(wrong_sr_files)}")
print("파일 경로:")
for f in wrong_sr_files:
    print(f)


샘플링 레이트가 16000이 아닌 파일 개수: 0
파일 경로:


In [ ]:
## 원본 데이터를 시각화


In [ ]:
import torch
import librosa
from datasets import Dataset
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# 1️⃣ Whisper Processor 로드 (한 번만 로드하면 됨)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="Korean",
    task="transcribe"
)

# 2️⃣ Feature extractor & tokenizer 따로 접근 가능
feature_extractor = processor.feature_extractor
tokenizer = processor.tokenizer

# 3️⃣ 오디오 로딩 + 전처리 함수
def preprocess_function(batch):
    # 오디오 파일 불러오기
    audio_path = f"/workspace/train/audio/{batch['audio_path']}"  # 경로 맞게 조정
    speech, sr = librosa.load(audio_path, sr=16000)  # Whisper expects 16kHz

    # Feature Extractor 적용
    inputs = feature_extractor(
        speech,
        sampling_rate=16000,
        return_tensors="pt"
    )

    # Tokenizer 적용
    with processor.as_target_processor():
        labels = processor.tokenizer(
            batch["text"],
            return_tensors="pt",
            padding="longest"
        )

    # 텐서 형태로 반환
    return {
        "input_features": inputs.input_features[0],
        "labels": labels.input_ids[0]
    }

In [ ]:
# 4️⃣ pandas DataFrame → HuggingFace Dataset 변환
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 5️⃣ 데이터셋에 전처리 적용
train_dataset = train_dataset.map(preprocess_function)
test_dataset = test_dataset.map(preprocess_function)
